In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import argparse
import onnxruntime as rt
import numpy as np
from inference_trt import init_decoder_inputs

In [2]:
import models
from inference import checkpoint_from_distributed, unwrap_distributed, load_and_setup_model, prepare_input_sequence
from tacotron2_common.utils import to_gpu, get_mask_from_lengths

In [ ]:
encoder = rt.InferenceSession('model_onnx/encoder.onnx')
decoder = rt.InferenceSession('model_onnx/decoder_iter.onnx')
postnet = rt.InferenceSession('model_onnx/postnet_2.onnx')
waveglow = rt.InferenceSession('model_onnx/waveglow.onnx')

2022-04-06 01:08:02.800146952 [W:onnxruntime:, graph.cc:119 MergeShapeInfo] Error merging shape info for output. 'memory' source:{1,,512} target:{0,mem_seq,Transposememory_dim_2}. Falling back to lenient merge.
2022-04-06 01:08:02.809808901 [W:onnxruntime:, graph.cc:119 MergeShapeInfo] Error merging shape info for output. 'processed_memory' source:{1,,128} target:{0,mem_seq,128}. Falling back to lenient merge.


In [4]:
# utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')

In [7]:
# print(encoder.get_inputs()[0].name)
# print(encoder.get_inputs()[1].name)
# print(encoder.get_outputs())
for out in postnet.get_outputs():
    print("this is the name of out put " + out.name)

NameError: name 'postnet' is not defined

In [12]:
# decoder_input = {}
for input in postnet.get_inputs():
    print("this is the name of in put " + input.name)
    

this is the name of in put mel_outputs


In [13]:
postnet.get_inputs()[0].name

'mel_outputs'

In [14]:
waveglow.get_inputs()[0].name

'mel'

In [8]:
text = "Hello world, I missed you so much."

In [9]:
## Encoder 
input_length = [50]
sequences, lengths = prepare_input_sequence([text],cpu_run = True)
sequences =  np.int64(sequences)
while len(sequences) < input_length[0]:
    sequences = np.append(sequences,0)
sequences = np.expand_dims(sequences,0)
memory, processed_memory, lens = encoder.run(["memory","processed_memory","lens"],{"sequences":sequences,"sequence_lengths":input_length})
memory, processed_memory, lens = torch.tensor(memory),torch.tensor(processed_memory),torch.tensor(lens)

In [11]:
## Decoder_iter
mel_lengths = np.zeros([memory.size(0)], dtype=np.int32)
not_finished = np.ones([memory.size(0)], dtype=np.int32)
mel_outputs, gate_outputs, alignments = (np.zeros(1), np.zeros(1), np.zeros(1))


(decoder_input, attention_hidden, attention_cell, decoder_hidden,
     decoder_cell, attention_weights, attention_weights_cum,
     attention_context, memory, processed_memory,
     mask) = init_decoder_inputs(memory, processed_memory, torch.tensor(input_length))

(decoder_input, attention_hidden, attention_cell, decoder_hidden,decoder_cell, attention_weights, attention_weights_cum,attention_context, memory, processed_memory,mask)  = decoder_input.numpy(), attention_hidden.numpy(), attention_cell.numpy(), decoder_hidden.numpy(),decoder_cell.numpy(), attention_weights.numpy(), attention_weights_cum.numpy(),attention_context.numpy(),memory.numpy(), processed_memory.numpy(),mask.numpy()

# decoder_output_name = ["decoder_output", "gate_prediction","out_attention_hidden", "out_attention_cell","out_decoder_hidden", "out_decoder_cell","out_attention_weights", "out_attention_weights_cum","out_attention_context"]
# decoder_input_element = {"decoder_input":decoder_input, "attention_hidden": attention_hidden, "attention_cell":attention_cell, "decoder_hidden":decoder_hidden ,"decoder_cell":decoder_cell, "attention_weights":attention_weights, "attention_weights_cum":attention_weights_cum,"attention_context":attention_context, "memory":memory, "processed_memory":processed_memory,"mask":mask}

# (mel_output, gate_output,
#              attention_hidden, attention_cell,
#              decoder_hidden, decoder_cell,
#              attention_weights, attention_weights_cum,
#              attention_context) = decoder.run(decoder_output_name , decoder_input_element)

# mel_outputs = np.expand_dims(mel_output, 2)
# gate_outputs = np.expand_dims(gate_output, 2)
# alignments = np.expand_dims(attention_weights, 2)

In [12]:
gate_threshold = 0.6
max_decoder_steps = 1000
first_iter = True

while True:
    decoder_output_name = ["decoder_output", "gate_prediction","out_attention_hidden", "out_attention_cell","out_decoder_hidden", "out_decoder_cell","out_attention_weights", "out_attention_weights_cum","out_attention_context"]
    decoder_input_element = {"decoder_input":decoder_input, "attention_hidden": attention_hidden, "attention_cell":attention_cell, "decoder_hidden":decoder_hidden ,"decoder_cell":decoder_cell, "attention_weights":attention_weights, "attention_weights_cum":attention_weights_cum,"attention_context":attention_context, "memory":memory, "processed_memory":processed_memory,"mask":mask}

    (mel_output, gate_output,
                 attention_hidden, attention_cell,
                 decoder_hidden, decoder_cell,
                 attention_weights, attention_weights_cum,
                 attention_context) = decoder.run(decoder_output_name , decoder_input_element)

    if first_iter:
        mel_outputs = np.expand_dims(mel_output, 2)
        gate_outputs = np.expand_dims(gate_output, 2)
        alignments = np.expand_dims(attention_weights, 2)
        first_iter = False
    else:
        mel_outputs = np.concatenate((mel_outputs, np.expand_dims(mel_output, 2)), 2)
        gate_outputs = np.concatenate((gate_outputs, np.expand_dims(gate_output, 2)), 2)
        alignments =np.concatenate((alignments, np.expand_dims(attention_weights, 2)), 2)
        
    dec = torch.le(torch.sigmoid(torch.Tensor(gate_output)), gate_threshold).to(torch.int32).squeeze(1)
    not_finished = not_finished*dec.numpy()
    mel_lengths += not_finished
    
    if np.sum(not_finished) == 0:
            print("Stopping after ",mel_outputs.size," decoder steps")
            break
    if mel_outputs.size == max_decoder_steps:
        print("Warning! Reached max decoder steps")
        break
        
    decoder_input = mel_output

Stopping after  15280  decoder steps


In [13]:
#postnet
mel_outputs_postnet = postnet.run(['mel_outputs_postnet'],{'mel_outputs':mel_outputs})

In [16]:
mel_outputs_postnet[0]

array([[[-7.406, -6.97 , -6.55 , ..., -7.613, -7.42 , -7.234],
        [-6.914, -6.492, -5.945, ..., -7.08 , -6.977, -6.832],
        [-6.484, -5.777, -5.15 , ..., -6.79 , -6.742, -6.63 ],
        ...,
        [-8.16 , -7.832, -7.516, ..., -9.37 , -9.484, -9.53 ],
        [-8.05 , -7.65 , -7.14 , ..., -9.47 , -9.53 , -9.56 ],
        [-8.11 , -7.562, -6.766, ..., -9.57 , -9.62 , -9.62 ]]],
      dtype=float16)

In [ ]:
# waveglow1 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
# waveglow1 = waveglow.remove_weightnorm(waveglow)
# waveglow1 = waveglow.to('cuda')
# waveglow1.eval()

In [ ]:
# torch.Tensor(mel_outputs_postnet[0]).to('cuda')

In [ ]:
# audio = waveglow.infer(torch.Tensor(mel_outputs_postnet[0]).to('cuda'))

In [17]:
for out in waveglow.get_outputs():
    print("this is the name of output " + out.name)

this is the name of output audio


In [18]:
for out in waveglow.get_inputs():
    print("this is the name of input " + out.name)

this is the name of input mel
this is the name of input z


In [19]:
stride = 256 # value from waveglow upsample
n_group = 8
z_size2 = (mel_outputs_postnet[0].shape[2]*stride)//n_group
z = np.random.randn(1, n_group, z_size2)

In [28]:
z_size2

11424

In [57]:
z.astype('float32')

array([[[ 1.1248934 , -0.07770122, -0.1793594 , ..., -0.25881702,
          0.23056704, -1.2464995 ],
        [-0.1231851 , -1.8648319 ,  0.4244355 , ..., -1.3905232 ,
         -1.1601046 ,  0.94064194],
        [-0.35573146,  0.1904765 ,  1.6952293 , ...,  0.7172618 ,
         -0.57208127,  1.1315285 ],
        ...,
        [ 0.18911465,  0.13937531,  0.26385745, ..., -0.03555681,
          0.23444729, -0.8888932 ],
        [ 0.33053353, -0.84941024, -1.014864  , ...,  0.21998811,
          2.0798995 , -0.6690167 ],
        [-0.80492204,  0.59472436, -1.4697658 , ..., -0.34563527,
         -0.5621281 ,  0.5709313 ]]], dtype=float32)

In [22]:
audio = waveglow.run(['audio'],{'mel':mel_outputs_postnet[0].astype('float32'),'z':z.astype('float32')})

Fail: [ONNXRuntimeError] : 1 : FAIL : Non-zero status code returned while running Conv node. Name:'Conv_1635' Status Message: CUDNN error executing cudnnConvolutionForward(s_.handle, &alpha, s_.x_tensor, s_.x_data, s_.w_desc, s_.w_data, s_.conv_desc, s_.algo, workspace.get(), s_.workspace_bytes, &beta, s_.y_tensor, s_.y_data)

In [36]:
np.array(z_size2).astype('float32')

array(11424., dtype=float32)

In [41]:
mel_outputs_postnet[0].shape

(1, 80, 357)

In [42]:
mel = torch.randn(1, 80, 620).cuda()

In [48]:
mel

tensor([[[-0.9599,  0.0870, -0.1175,  ...,  0.0625,  0.6903,  0.5833],
         [ 1.9285,  2.8344,  1.1614,  ..., -0.6740,  1.6322, -1.0374],
         [ 0.4578, -1.0032, -0.9797,  ..., -0.1885, -1.5935, -0.8414],
         ...,
         [-0.4867, -1.7212, -0.9229,  ..., -0.9055,  0.5610, -0.6816],
         [ 0.0876, -0.5592,  0.4963,  ...,  1.1476, -0.8665, -0.5151],
         [-1.1171, -1.2848, -1.2479,  ..., -1.1778, -0.3264,  0.2585]]],
       device='cuda:0')

In [49]:
np.random.randn(1, n_group, z_size2)

array([[[-0.00336026, -1.97647186, -1.49949022, ...,  0.68223006,
         -2.01466388, -0.64972934],
        [ 0.0941771 ,  0.04563647, -1.03623937, ...,  1.35380583,
          0.67739   ,  0.71338536],
        [ 0.16346079,  0.84346013, -0.28859739, ...,  1.2560419 ,
          0.34930573,  0.44432217],
        ...,
        [-0.05122804, -0.76199988,  0.23117744, ...,  0.54678478,
         -0.12461607, -0.04727315],
        [ 0.65287573,  1.86149349,  0.70598217, ...,  1.01392168,
         -1.78594388,  0.76104792],
        [-0.7292311 ,  0.29504124, -0.36263551, ...,  0.38968304,
          0.64951391, -0.26704558]]])